In [1]:
from pyautocad import Autocad, APoint, distance, aDouble
from math import pi, atan2, degrees, floor, ceil, sqrt
import pandas as pd
import numpy as np
#Find file name "acad_aag.chm - ActiveX and VBA Developer’s Guide"
#Find file name "acadauto.chm - ActiveX and VBA Reference"
#@ Loaction -> "C:\Program Files\Common Files\Autodesk Shared"

# Mark the building

In [3]:
def prepare_dataframe(name_df: str, x: list, y: list) -> pd.DataFrame:
    """
    Prepare dataframe the columes are x, y, name
    
    Args:
        df_nmae (str): name of dataframe
        x (list): x coordinate of mark
        y (list): y coordinate of mark
        
    Returns:
        df_name (pd.DataFrame): prepared dataframe 
    """
    name_df = pd.DataFrame()
    name_df['x'] = x
    name_df['y'] = y

    Name = []
    for N in range(0,len(name_df)):
        Name.append("Point_"+str(N+1))
        
    name_df['Name'] = Name
    
    return name_df

In [4]:
def execute_line_polygon(df: pd.DataFrame,
                         col_x: str='x', 
                         col_y:str ='y')-> object:
    """add polyline polygon from the Mark

    Args:
        df (pd.DataFrame): The Dataframe contains x,y coordinate of the polygon
        col_x (str): The column name of x coordinate to plot. Defaults to 'x'.
        col_y (str): The column name of y coordinate to plot. Defaults to 'y'.
        
    Returns:
        polygond (object): The object of the polygon in AutoCAD
    """
    plot = []
    for poly in range(0,len(df)):
        plot.append(df[col_x][poly])
        plot.append(df[col_y][poly])
        plot.append(0)

        
    plot.append(df[col_x][0])
    plot.append(df[col_y][0])
    plot.append(0) 

    polygon = aDouble(plot)

    #Execute Line 
    polygond = acad.model.AddPolyline(polygon)
    
    return polygond

In [5]:
#Input Coordinate 

x = [743401.40,
743402.10,
743404.93,
743405.18,
743454.54,
743454.74,
743467.68,
743467.52,
743469.81,
743468.70,
743415.20,
743414.78,
]
y = [1402964.44,
1402984.25,
1402984.17,
1402992.47,
1402989.46,
1402991.24,
1402990.24,
1402984.04,
1402983.29,
1402972.99,
1402977.62,
1402962.78,
]

In [6]:
# Prepare Dataframe of Mark
Mark = prepare_dataframe('Mark', x, y)
Mark

,x,y,Name
0,743401.40,1402964.44,Point_1
1,743402.10,1402984.25,Point_2
2,743404.93,1402984.17,Point_3
3,743405.18,1402992.47,Point_4
4,743454.54,1402989.46,Point_5
5,743454.74,1402991.24,Point_6
6,743467.68,1402990.24,Point_7
7,743467.52,1402984.04,Point_8
8,743469.81,1402983.29,Point_9
9,743468.70,1402972.99,Point_10


In [7]:
#Create Building Drawing as per input
polygond = execute_line_polygon(Mark)

# The Drawing should be shown in AutoCAD after running this code

# Rectangularized the Building

In [8]:
def rotate_polygon(df:pd.DataFrame, polygond:object)-> object:
    "TODO: Generalize the Ref by calculating two consecutive points that has the longest distance"
    """Rotate polygon as per Marker
    
    Args:
        Mark (pd.DataFrame): The Dataframe contains x,y coordinate of the polygon
        polygond (object): The object of polygon in AutoCAD
        
    Returns:
        polygond (object): The object of rotated polygon in AutoCAD
    """

    #Input Reference to Rotate make SN axis
    Ref_1 = 'Point_10' 
    Ref_2 = 'Point_11'
    xaxis = df.x[9] - df.x[10]
    yaxis = df.y[9] - df.y[10]
    degree = degrees(atan2(yaxis,xaxis))
    degree = -degree

    #Rotate Building as per Marker
    Point =  APoint(df.x[10],df.y[10])
    rotated_polygond = polygond.Rotate(Point,pi*degree/180)
    
    return rotated_polygond

In [9]:
def record_coordinate_of_polygon()-> pd.DataFrame:
    """Record coordinate of polygon

    Args:
        None: The object of polygon in AutoCAD
        
    Returns:
        After_Rotate (pd.DataFrame): The Dataframe contains x,y coordinate of the rotated polygon 
    """
    x_rot = []
    y_rot = []
    for item in acad.iter_objects("AcDb2dPolyline"):
        for n in range(0,12):
            cor = list(item.Coordinate[n])
            x_rot.append(cor[0])
            y_rot.append(cor[1])
            
    After_Rotate = prepare_dataframe('After_Rotate', x_rot, y_rot)
    
    return After_Rotate

In [10]:
def calculate_axis_of_each_point(df: pd.DataFrame, 
                                 col_x: str= 'x',
                                 col_y: str= 'y')-> pd.DataFrame:
    """Calculate Axis of each point (In Advance building design may be used)

    Args:
        df (pd.DataFrame): The DataFrame of polygon

    Returns:
        df (pd.DataFrame): The DataFrame of polygon with degree of each point
    """
    
    deg = []
    for d in range(1,len(df)):
        xaxis = df[col_x][d] - df[col_x][d-1]
        yaxis = df[col_y][d] - df[col_y][d-1]
        degree = abs(degrees(atan2(yaxis,xaxis)))
        deg.append(degree)
        
    xaxis = df[col_x][0] - df[col_x][len(df)-1]
    yaxis = df[col_y][0] - df[col_y][len(df)-1]
    degree = abs(degrees(atan2(yaxis,xaxis)))
    deg.append(degree)
    df['deg'] = deg
    
    return df

In [11]:
def calculate_difference_length_of_each_point(df: pd.DataFrame, 
                                              col_x: str= 'x',
                                              col_y: str= 'y')-> pd.DataFrame:
    """Calculate Difference Legnth of each point 

    Args:
        df (pd.DataFrame): The DataFrame of polygon

    Returns:
        df (pd.DataFrame): The DataFrame of polygon with difference length of each point
    """
    d_x = []
    d_y = []

    for d in range(0,len(df)-1):
        d_x.append(abs(df[col_x][d+1]-df[col_x][d]))
        d_y.append(abs(df[col_y][d+1]-df[col_y][d]))
        
    d_x.append(abs(df[col_x][0]-df[col_x][len(df)-1]))
    d_y.append(abs(df[col_y][0]-df[col_y][len(df)-1]))
    df["d_x"] = d_x
    df["d_y"] = d_y
    
    return df

In [12]:
def rectangularized_the_building(df: pd.DataFrame, 
                                 col_x: str= 'x',
                                 col_y: str= 'y')-> pd.DataFrame:
    """Rectangularized the building

    Args:
        df (pd.DataFrame): The DataFrame of polygon

    Returns:
        df (pd.DataFrame): The DataFrame of polygon with x,y coordinate of each point properly
    """
    #To adjust building coordinate to be should be building sizing 
    # (Have to add-on f(x) of not reatangular building)
    Nx = []
    Ny = []
    Nx.append(df[col_x][0])
    Ny.append(df[col_y][0])
    for a in range(0,len(df)-2):
        if df['d_x'][a] < df['d_y'][a]:
            Nx.append(df[col_x][a])
            Ny.append(df[col_y][a+1])
        else:
            Nx.append(df[col_x][a+1])
            Ny.append(df[col_y][a])

    if df['d_x'][len(df)-1] < df['d_y'][len(df)-1]:
        Nx.append(df[col_x][0])
        Ny.append(df[col_y][len(df)-1])
    else:
        Nx.append(df[col_x][len(df)-2])
        Ny.append(df[col_y][0])
        
    Proper_Polygon = prepare_dataframe('Proper_Polygon', Nx, Ny)
    
    return Proper_Polygon

In [13]:
def prepare_dataframe_of_proper_building(df:pd.DataFrame)-> pd.DataFrame:
    """Prepare dataframe of proper building

    Args:
        df (pd.DataFrame): The DataFrame of rotated polygon

    Returns:
        df (pd.DataFrame): The DataFrame of rotated polygon with additional columns
    """
    
    df = calculate_axis_of_each_point(df)
    df = calculate_difference_length_of_each_point(df)
    df = rectangularized_the_building(df)
    
    return df

In [14]:
polygond = rotate_polygon(Mark, polygond)
# The Drawing should be rotated in AutoCAD after running this code ot have the longest length of the building in SN axis

In [15]:
After_Rotate = record_coordinate_of_polygon()
After_Rotate

,x,y,Name
0,743402.587766,1.402963e+06,Point_1
1,743401.577145,1.402983e+06,Point_2
2,743404.403504,1.402983e+06,Point_3
3,743403.936949,1.402992e+06,Point_4
4,743453.372661,1.402993e+06,Point_5
5,743453.418445,1.402995e+06,Point_6
6,743466.396478,1.402995e+06,Point_7
7,743466.771637,1.402989e+06,Point_8
8,743469.117774,1.402988e+06,Point_9
9,743468.899971,1.402978e+06,Point_10


In [16]:
Proper_Polygon = prepare_dataframe_of_proper_building(After_Rotate)
Proper_Polygon

,x,y,Name
0,743402.587766,1.402963e+06,Point_1
1,743402.587766,1.402983e+06,Point_2
2,743404.403504,1.402983e+06,Point_3
3,743404.403504,1.402992e+06,Point_4
4,743453.372661,1.402992e+06,Point_5
5,743453.372661,1.402995e+06,Point_6
6,743466.396478,1.402995e+06,Point_7
7,743466.396478,1.402989e+06,Point_8
8,743469.117774,1.402989e+06,Point_9
9,743469.117774,1.402978e+06,Point_10


# Delete non-rectangulatized Building

In [17]:
def delete_polygon()-> object:
      """Deleted Prelimary Input 

      Args:
          None: The object of polygon in AutoCAD

      Returns:
          None: The object of polygon in AutoCAD that has been deleted
      """
      for item in acad.iter_objects():
            item.Delete()

In [18]:
delete_polygon()
# The Drawing should be deleted in AutoCAD after running this code

# Create proper Building

In [20]:
polygond = execute_line_polygon(Proper_Polygon, 'x', 'y')
# The Drawing should be shown in AutoCAD after running this code

# Verify that it is rectangularized

In [21]:
#Just Calculate Cross check building
Proper_Polygon = calculate_axis_of_each_point(Proper_Polygon,'x', 'y')
Proper_Polygon = calculate_difference_length_of_each_point(Proper_Polygon, 'x', 'y')
Proper_Polygon
# If the drawing is rectangularized, the degree of each point should be 90 degree

,x,y,Name,deg,d_x,d_y
0,743402.587766,1.402963e+06,Point_1,90.0,0.000000,19.796584
1,743402.587766,1.402983e+06,Point_2,0.0,1.815738,0.000000
2,743404.403504,1.402983e+06,Point_3,90.0,0.000000,8.454947
3,743404.403504,1.402992e+06,Point_4,0.0,48.969157,0.000000
4,743453.372661,1.402992e+06,Point_5,90.0,0.000000,3.047633
5,743453.372661,1.402995e+06,Point_6,0.0,13.023817,0.000000
6,743466.396478,1.402995e+06,Point_7,90.0,0.000000,6.071299
7,743466.396478,1.402989e+06,Point_8,0.0,2.721296,0.000000
8,743469.117774,1.402989e+06,Point_9,90.0,0.000000,10.907112
9,743469.117774,1.402978e+06,Point_10,180.0,53.917774,0.000000


# Area Zone Division by Y-Axis

In [22]:
def divide_zone_as_rectangles(Proper_Polygon: pd.DataFrame)-> object:
    """Divide zone as rectangles

    Args:
        Proper_Polygon (pd.DataFrame): The DataFrame of proper polygon prepared to be divided

    Returns:
        object: The object of the divided in zone polygon in AutoCAD
        zone (list): The list of 3-tuples of each zone
        zone_length (list): The list of length of each zone
    """
    
    # create empty list to store list of 3-tuples of each zone
    zone_list = []
    # create empty list to store length of each zone
    zone_length = []
    
    # Step1: list all y of the polygon, then let the lowest y be the bottom of the rectangle, 
    all_y = list(Proper_Polygon['y'])
    y_sort = list(set(all_y))
    y_sort = sorted(y_sort)

    # create empty list to store x of the previous zone prepare to analyze the next zone
    x_previous = []

    # loop for each y_base
    for i in range (len(y_sort)-1): # len(y_sort)-1 because the highest y is the top of the building
        y_base = y_sort[i] # y_base is the lowest y 
    
    # Step2: find x corresponding to that y_base
        # include x_previous to analyze new x in the next zone
        if len(x_previous) == 0: # at the first zone, x_previous is empty
            x_sample = list(Proper_Polygon['x'][Proper_Polygon['y'] == y_base])
            
        elif len(x_previous) != 0: # at the next zone, x_previous is not empty
            x_sample = list(Proper_Polygon['x'][Proper_Polygon['y'] == y_base])
            x_sample.extend(x_previous)
            
        # Note that: x that is going to be selected must has y corresponding which is higher than y_base
        # create a dataframe to check x and y of the polygon
        df_analyze = []
        for x in x_sample:
            x_col = Proper_Polygon['x'] == x
            y_col = Proper_Polygon['y'] > y_base
            df = Proper_Polygon.loc[x_col & y_col]
            df_analyze.append(df)
        df_analyze = pd.concat(df_analyze)

        x_analyze = list(df_analyze['x'])
        left_x = min(x_analyze)
        right_x = max(x_analyze)

    # Step3: find y_top which is the top of the rectangle, which is the lower y corresponding to left_x and right_x
        y_sample = list(Proper_Polygon['y'][Proper_Polygon['x'] == left_x])
        y_sample2 = list(Proper_Polygon['y'][Proper_Polygon['x'] == right_x])
        y_sample.extend(y_sample2) 
        # select only y that is higher than y_base
        y_sample = [y for y in y_sample if y > y_base]
        # y_top is y at the top of the rectangle
        y_top = min(y_sample)
        y_higher_top = max(y_sample) # higher y is not used

    # Step4: select x at the top of the rectangle
        # If x is on the edge of the polygon, x_select is that x
        for x in x_analyze:
            # x_select is always at the edge of the polygon
            try:
                if x not in list(Proper_Polygon['x'][Proper_Polygon['y'] == y_top]):
                    x_select = x
                    # store the new x to x_previous list to analyze the next x for the next zone
                    x_previous.append(x_select)
            except:
                # If x is not on the edge, there is no error
                continue
     
    # Step5: execute polyline polygon from the Mark
        try:
            if x_select < right_x:
                x_to_plot = [left_x, right_x, right_x, x_select]
            elif x_select > left_x:
                x_to_plot = [left_x, right_x, x_select, left_x]
        # if x is not on the edge, use that left_x and right_x to plot
        except:
            x_to_plot = [left_x, right_x, right_x, left_x]

        y_to_plot = [y_base, y_base, y_top, y_top]

        # Record 3-tuples of each zone
        z = [] # z is the 3-tuples of each zone
        
        # The order is as following
        # left bottom, right bottom, right top, left top
        for ii in range(len(x_to_plot)):
            z.append(x_to_plot[ii])
            z.append(y_to_plot[ii])
            z.append(0)
        zone_list.append(z)
        
        # Record length of each zone
        try:
            if x_select < right_x:
                zone_length.append(right_x - x_select)
            elif x_select > left_x:
                zone_length.append(x_select - left_x)
        except:
                zone_length.append(right_x - left_x)
        
        # create a dataframe named Mark to execute line of the polygon that has been divided
        # Mark = prepare_dataframe('Mark', x_to_plot, y_to_plot)
        # execute_line_polygon(Mark, 'x', 'y')
        print(f"zone {i+1} is done")
    
    return zone_list, zone_length

In [23]:
zone,zone_length = divide_zone_as_rectangles(Proper_Polygon)

zone 1 is done
zone 2 is done
zone 3 is done
zone 4 is done
zone 5 is done


# Area Combination by Percentile

In [24]:
# Function to Combine Zone Area

# create empty list to store difference of length
zone_length_diff = []
for i in range(1,len(zone_length)): # loop for each zone starting from the 2nd zone
    diff = zone_length[i]-zone_length[i-1] # calculate difference of length
    zone_length_diff.append(diff) 
    
# calulcate range to seperate
q1, q3 = np.percentile(zone_length_diff,[25,75])


combine_zone = [] # create empty list to store combined zone
_continue = False # create boolean to check whether the zone is in range Q1 and Q3
c = 0 # create counter to count zone

for d in zone_length_diff:
    
    # If difference of length is in the range of Q1 and Q3, then combine zone
    if q1 < d and d < q3: 
        
        print("Combine Zone n with n+1")
        if _continue == True: #Other Zone Analyze
            xmin = max(combine_zone[-1][0],zone[c][0])
            xmax = min(combine_zone[-1][3],zone[c][3])
            ymin = min(combine_zone[-1][1],zone[c][1])
            ymax = max(combine_zone[-1][7],zone[c][7])
            z = 0
            com = [xmin,ymin,z,xmax,ymin,z,xmax,ymax,z,xmin,ymax,z]
            combine_zone.pop(-1)
            combine_zone.append(com)
            c += 1
            #print(c)
        
        # analyze 1st zone
        if _continue == False: 
            xmin = max(zone[c][0],zone[c+1][0])
            xmax = min(zone[c][3],zone[c+1][3])
            ymin = min(zone[c][1],zone[c+1][1])
            ymax = max(zone[c][7],zone[c+1][7])
            z = 0
            com = [xmin,ymin,z,xmax,ymin,z,xmax,ymax,z,xmin,ymax,z]
            combine_zone.append(com)
            _continue = True
            c += 2  
             
    else: #Zone is in range Q1 and Q3
        print("Zone n is dependent")
        combine_zone.append(zone[c])
        _continue = False
        #print(c)
        c += 1
        


Zone n is dependent
Combine Zone n with n+1
Combine Zone n with n+1
Zone n is dependent


# Calculate Area after Combination

In [25]:
# Calculate Area After Combine
x_zone = [] # record x-axis length of each zone
y_zone = [] # record y-axis length of each zone
area_zone = [] # record area of each zone

for zonearea in range(0,len(combine_zone)):
    
    x_axis_length = combine_zone[zonearea][3] - combine_zone[zonearea][0] 
    y_axis_length = combine_zone[zonearea][7] - combine_zone[zonearea][1] 
    area = x_axis_length*y_axis_length
    area_zone.append(area)
    x_zone.append(x_axis_length)
    y_zone.append(y_axis_length)
    
area_zone,x_zone, y_zone   

([180.6166820815404, 863.6103817173943, 39.691813648183114],
 [12.612233967054635, 61.992973731132224, 13.023817056207918],
 [14.32075257669203, 13.930778437294066, 3.047632923349738])

# Design Solar Layout

In [26]:
num_pv_per_string = 18
num_inverter = 100/(num_pv_per_string/2)
devide = num_inverter-floor(num_inverter)
devide,num_inverter

(0.11111111111111072, 11.11111111111111)

In [27]:
# spec pv and length by standards
pv_width = 1134/1000
pv_length = 2278/1000

pv_gap = 15/1000
pv_column_gap = 400/1000

building_edge = 1000/1000

walkway_width = 400/1000
walkway_endclamp = 100/1000
walkway_pv_gap = 100/1000

In [28]:
num_pv_per_string = 18

In [29]:
#Find main axis
x_axis_index = 9999
y_axis_index = 9999
if max(x_zone) > max(y_zone):
    x_axis_index = x_zone.index(max(x_zone))
else:
    y_axis_index = y_zone.index(max(y_zone))
    
x_axis_index,y_axis_index  

(1, 9999)

In [30]:
#Find Suitable WalkWay
#Partial Analyze based on sub-axis



In [31]:
#Calculate PV per column (Big Area)
# 1 tier is PV per column == Invertor string
# 2 tier is PV per column == (Invertor string)/2
# 3 tier is others



In [32]:
Building_side = 2
Req_Main_Walk = walkway_endclamp + walkway_width + 3*walkway_endclamp + Building_side*building_edge
Req_Pv_String = pv_width*num_pv_per_string + pv_gap*(num_pv_per_string-1)

In [33]:
Req_Distance = Req_Main_Walk + Req_Pv_String
Req_Distance

23.467

In [34]:
# #String Number Cal
# No_string = (y_axis_length-Req_Main_Walk)/Req_Pv_String
# No_string

In [35]:
#PV number
No_PV = (y_zone[1]-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
No_PV = floor(No_PV)
No_PV                

9

# PV Layout in Biggest Zone

In [36]:
#Start from upper

pv = []
#1st Panel

#1st point in 1st panel
# "x" (left) + building edge
x = combine_zone[1][0] + building_edge 
# "y" (From above) - building edge, walk way, cable tray, life line & water line
y = combine_zone[1][7] - building_edge - walkway_width - 3*walkway_endclamp 
z = 0
edge = [x,y,z]
pv.extend(edge)

#2nd point in 1st panel
x = x + pv_length
y = y
z = 0
edge = [x,y,z]
pv.extend(edge)

#3rd point in 1st panel
x = x 
y = y - pv_width
z = 0
edge = [x,y,z]
pv.extend(edge)

#4th point in 1st panel
x = combine_zone[1][0] + building_edge 
y = combine_zone[1][7] - building_edge - walkway_width - 3*walkway_endclamp - pv_width
z = 0
edge = [x,y,z]
pv.extend(edge)

#Cover Rectangular point in 1st panel
x = combine_zone[1][0] + building_edge  #Back to start point
y = combine_zone[1][7] - building_edge - walkway_width - 3*walkway_endclamp #Back to start point
z = 0
edge = [x,y,z]
pv.extend(edge)

spv = aDouble(pv)
Panel = acad.model.AddPolyline(spv)

In [37]:
#PV string column

No_column_set = 2 
x_1st = combine_zone[1][0] + building_edge  # Start point
x_nth = combine_zone[1][3] - building_edge  # Last point

PV_set = ((x_nth-x_1st) + (walkway_width+2*walkway_pv_gap))/((pv_length*No_column_set + pv_gap*(No_column_set-1)) + (walkway_width+2*walkway_pv_gap))

Main_PV_set = floor(PV_set)
Portion_PV_set = (PV_set - Main_PV_set)*(pv_length*No_column_set+pv_gap*(No_column_set-1)+walkway_width+2*walkway_pv_gap)/(pv_length+pv_gap)
Portion_PV_set = floor(Portion_PV_set)

PV_set = No_column_set*Main_PV_set+Portion_PV_set
PV_set

23

In [38]:
#P2 = Panel.Copy()
no_B = 0 
g = 0
x = combine_zone[1][0] + building_edge  #Back to start point
y = combine_zone[1][7] - building_edge - walkway_width - 3*walkway_endclamp #Back to start point
Round = 0

# create empty list to store value of edge of x and y to find contact point of DC cables
x_edge_list = []
y_edge_list = []

for c in range(0,PV_set):
    
    if (c>0) & (c%No_column_set==0):
        no_B += 1
    if (c>0) & (Round>No_column_set):
        Round = 1
    if (c>0) & (Round<No_column_set):
        g +=1
        
    
    print(f"c: {c} Round: {Round} g: {g}")
    Round += 1
         
    xi = x+(c*pv_length)+(g*pv_gap)+no_B*(2*walkway_pv_gap+walkway_width)
    P2 = Panel.Copy()
    P2.Move(APoint(x,y),
            APoint(xi,y)) #Plus in "C" from "left" to "Right"

    
    for r in range(1,No_PV):
        yi = y - pv_width*r - pv_gap*r
        P2 = Panel.Copy()
        P2.Move(APoint(x,y),
            APoint(xi,yi)) #Minus in "r" from "Up" to "Down"
    x_edge = xi 
    y_edge = (y + (yi - pv_width))/2
    
    x_edge_list.append(x_edge)
    y_edge_list.append(y_edge)
    #if (c>0) & (c%No_column_set==0):
    #    no_B += 1

c: 0 Round: 0 g: 0
c: 1 Round: 1 g: 1
c: 2 Round: 2 g: 1
c: 3 Round: 1 g: 2
c: 4 Round: 2 g: 2
c: 5 Round: 1 g: 3
c: 6 Round: 2 g: 3
c: 7 Round: 1 g: 4
c: 8 Round: 2 g: 4
c: 9 Round: 1 g: 5
c: 10 Round: 2 g: 5
c: 11 Round: 1 g: 6
c: 12 Round: 2 g: 6
c: 13 Round: 1 g: 7
c: 14 Round: 2 g: 7
c: 15 Round: 1 g: 8
c: 16 Round: 2 g: 8
c: 17 Round: 1 g: 9
c: 18 Round: 2 g: 9
c: 19 Round: 1 g: 10
c: 20 Round: 2 g: 10
c: 21 Round: 1 g: 11
c: 22 Round: 2 g: 11


# Create Main Walkway

In [39]:
#Walk Way create (Main)
#Start from lower
Main_way = []
z = 0
x = combine_zone[1][0] + building_edge  #Start point
y = combine_zone[1][1] + building_edge  #Lower Main Edge
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = combine_zone[1][3] - building_edge  
y = y  
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = x  
y = y + walkway_width #Upper Main Edge
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = combine_zone[1][0] + building_edge  
y = y 
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = x  
y = y - walkway_width  
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

mw = aDouble(Main_way)
MainWay = acad.model.AddPolyline(mw)
MainWay.Color = 2



# Create DC Line

In [40]:
x_contact_point_list = []
y_contact_point_list = []
if len(x_edge_list)%2 == 0:
    for i in range (0,len(x_edge_list),2):
        x = (x_edge_list[i] + (x_edge_list[i+1] + pv_length))/2
        y = y_edge_list[i]
        x_contact_point_list.append(x)
        y_contact_point_list.append(y)        
        
elif len(x_edge_list)%2 != 0:
    for i in range (0,len(x_edge_list)-1,2):
        x = (x_edge_list[i] + (x_edge_list[i+1] + pv_length))/2
        y = y_edge_list[i]
        x_contact_point_list.append(x)
        y_contact_point_list.append(y)
    x = x_edge_list[-1]
    y = y_edge_list[-1]
    x_contact_point_list.append((x+x+pv_length)/2) 
    y_contact_point_list.append(y)


In [41]:
inverter_coor = [743467.7324,1402984.1785]

In [42]:
# create dc line from contact point to inverter
y_center_main_way = (Main_way[1]+Main_way[7])/2
dc_length_list = []
for i in range (len(x_contact_point_list)):
    x1 = x_contact_point_list[i]
    y1 = y_contact_point_list[i]
    p1 = APoint(x1,y1)
        
    x2 = x1
    y2 = y_center_main_way
    p2 = APoint(x2,y2)
        
    x3 = inverter_coor[0]
    y3 = y_center_main_way
    p3 = APoint(x3,y3)
        
    x4 = x3
    y4 = inverter_coor[1]
    p4 = APoint(x4,y4)

    line1 = acad.model.AddLine(p1,p2)
    line1.Color = 1
    line2 = acad.model.AddLine(p2,p3)
    line2.Color = 1
    line3 = acad.model.AddLine(p3,p4)
    line3.Color = 1
    
    length_part_1 = distance(p1,p2)
    length_part_2 = distance(p2,p3)
    length_part_3 = distance(p3,p4)
    
    dc_length = length_part_1 + length_part_2 + length_part_3
    dc_length_list.append(dc_length)

In [43]:
 dc_length_list

[71.26967399811838,
 66.0986739982618,
 60.927673998172395,
 55.75667399808299,
 50.58567399822641,
 45.41467399825342,
 40.24367399828043,
 35.07267399819102,
 29.901673998101614,
 24.730673998245038,
 19.55967399815563,
 15.535173998214304]

In [44]:
dc_length_total = sum(dc_length_list)
dc_length_total

515.0965879783034

# Voltage Drop Calculation

Vdrop (V) = Iwire (A) × Rwire(Ω)
          = Iwire (A) × (2 × L(m) × Rwire (Ω/km) / 1000(m/km))

Vdrop (V) = num_PV x Voltage of each PV (in 1 string)


In [45]:
# Assume that this project use 4 mm2 DC cable

Iwire = 14.47 #unit: A
Rwire = 5.09
voltage_drop_list = []
for l in dc_length_list:
    r = 2*l*Rwire/1000 #unit: ohm
    voltage_drop = Iwire * r
    voltage_drop_list.append(voltage_drop)

In [46]:
voltage_drop_list

[10.49835082042323,
 9.736638733844355,
 8.974926647231186,
 8.213214560618015,
 7.451502474039144,
 6.689790387443121,
 5.928078300847099,
 5.166366214233929,
 4.404654127620759,
 3.642942041041886,
 2.8812299544287163,
 2.2884025917373587]

In [47]:
No_PV

9

In [48]:
voltage_at_start_terminal = No_PV * 51.02

# Create Branch

In [49]:
#Walk Way create (Branch)
branch_way = []
x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)


x = x + walkway_width  #right point
y = y #right point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x #right bottom point
y = combine_zone[1][1] + building_edge + walkway_width   #right bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x - walkway_width #left bottom point
y = y  #left bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

bw = aDouble(branch_way)
BranchWay = acad.model.AddPolyline(bw)
BranchWay.Color = 2

In [50]:
#Expand branch line

Pre_Walk = No_column_set*pv_length+(No_column_set-1)*pv_gap+walkway_pv_gap
Back_Walk = walkway_pv_gap + walkway_width
x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
Round = 0
pv = 0
m_point = Pre_Walk*(Round+1) + Round*Back_Walk
BranchWay.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"
x = x+m_point

#bw2 = BranchWay.Copy()
if PV_set%No_column_set!=0:
    PV_set_Round = PV_set-1
if PV_set%No_column_set==0:
    PV_set_Round = PV_set
    
while pv < PV_set_Round:
    #print("pv : "+str(pv))
    bw2 = BranchWay.Copy()
    m_point = (2*walkway_pv_gap + walkway_width + No_column_set*pv_length + (No_column_set-1)*pv_gap)*Round
    #m_point = Pre_Walk*(Round+1) + Round*Back_Walk
    pv = pv + No_column_set
    Round += 1
    
    bw2.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"
    
    

# PV Layout in 1st Zone

In [51]:
#Analyzing Length to Plan
x_axis_length = combine_zone[0][3] - combine_zone[0][0]
y_axis_length = combine_zone[0][7] - combine_zone[0][1]
area = x_axis_length*y_axis_length
area_zone.append(area)
x_zone.append(x_axis_length)
y_zone.append(y_axis_length)

In [52]:
combine_zone[0][3], combine_zone[0][0]

(743415.2, 743402.5877660329)

In [53]:
combine_zone[0][7], combine_zone[0][1]

(1402977.62, 1402963.2992474234)

In [54]:
combine_zone[0][7], combine_zone[1][1] + building_edge - walkway_endclamp

(1402977.62, 1402978.52)

In [55]:
Lower_Main_Way = combine_zone[1][1] + building_edge - walkway_endclamp
Upper_Area_Edge = combine_zone[0][7]

if Lower_Main_Way > Upper_Area_Edge:
    PV_Upper_Area_2_Edge = Lower_Main_Way
    
PV_Upper_Area_2_Edge
PV_Lower_Area_2_Edge = combine_zone[0][1]

No_PV = (PV_Upper_Area_2_Edge - PV_Lower_Area_2_Edge + pv_gap)/(pv_width+pv_gap)
No_PV = floor(No_PV)
No_PV 

13

In [56]:
#Calculate PV column in new area
st_x_Ref = combine_zone[1][0] + building_edge #1st Edge of PV Panel
x_nth = combine_zone[0][3] - building_edge  #Area 2 Edge for compare
g = 0
g1 = 0
w = 0
w1 = 0
for r in range(1,PV_set):
    
    if r > 2:
        
        if g1 > 1:
            g+=1
            
        if g1 == 2:
            g1 = 0
            
        g1 += 1
        
    if r == 2:
        g  += 1
        g1 += 1
        
    
        
    if r > No_column_set+1:
        
        if w1 > No_column_set-1:
            w += 1
        
        if w1 == No_column_set:
            w1 = 0
            
        w1 += 1
        
    if r == No_column_set+1:
        w += 1
        w1 += 1

    Ref_PV_edge_compare = st_x_Ref + r*pv_length + g*pv_gap + w*(walkway_width + 2*walkway_pv_gap)
    #print("r : "+str(r)+" w : "+str(w)+" w1 : "+str(w1))
    #print("r : "+str(r)+" g : "+str(g)+" w : "+str(w))
    
    if Ref_PV_edge_compare > x_nth:
        pv = r-1
        break

pv

3

In [57]:
#P2 = Panel.Copy()
PV_set = pv
no_B = 0
g = 0
x = combine_zone[1][0] + building_edge  #1st Edge of PV Panel
y = combine_zone[1][7] - building_edge - walkway_width - 3*walkway_endclamp #Back to start point
yi = Lower_Main_Way
Round = 0

# create empty list to store value of edge of x and y to find contact point of DC cables
x_edge_list_1 = []
y_edge_list_1 = []

for c in range(0,PV_set):
    
    if (c>0) & (c%No_column_set==0):
        no_B += 1
        
    if (c>0) & (Round>No_column_set):
        Round = 1
    if (c>0) & (Round<No_column_set):
        g +=1
        
    
    print("c : "+str(c)+" Round : "+str(Round)+" g : "+str(g))
    Round += 1
         
    xi = x+(c*pv_length)+(g*pv_gap)+no_B*(2*walkway_pv_gap+walkway_width)    
    P2 = Panel.Copy()
    P2.Move(APoint(x,y),
            APoint(xi,yi)) #Plus in "C" from "left" to "Right"

    for r in range(1,No_PV):
        
        y_new = yi-pv_width*r-pv_gap*r
        P2 = Panel.Copy()
        P2.Move(APoint(x,y),
            APoint(xi,y_new)) #Minus in "r" from "Up" to "Down"
    x_edge = xi 
    y_edge = (yi + y_new - pv_width)/2 
    x_edge_list_1.append(x_edge)
    y_edge_list_1.append(y_edge)      
    #if (c>0) & (c%No_column_set==0):
    #    no_B += 1
    


c : 0 Round : 0 g : 0
c : 1 Round : 1 g : 1
c : 2 Round : 2 g : 1


In [59]:
x_contact_point_list_1 = []
y_contact_point_list_1 = []
if len(x_edge_list_1)%2 == 0:
    for i in range (0,len(x_edge_list_1),2):
        x = (x_edge_list_1[i] + x_edge_list_1[i+1] + pv_length)/2
        y = y_edge_list_1[i]
        x_contact_point_list_1.append(x)
        y_contact_point_list_1.append(y)
        
elif len(x_edge_list_1)%2 != 0:
    for i in range (0,len(x_edge_list_1)-1,2):
        x = (x_edge_list_1[i] + x_edge_list_1[i+1] + pv_length)/2
        y = y_edge_list_1[i]
        x_contact_point_list_1.append(x)
        y_contact_point_list_1.append(y)
    x_contact_point_list_1.append((x_edge_list_1[-1] + (x_edge_list_1[-1]+pv_length))/2)
    y_contact_point_list_1.append(y)

# Create DC Cables

In [60]:
# create dc line from contact point to inverter
y_center_main_way = (Main_way[1]+Main_way[7])/2
plot = []
for i in range (len(x_contact_point_list_1)):
    x1 = x_contact_point_list_1[i]
    y1 = y_contact_point_list_1[i]
    p1 = APoint(x1,y1)
    
    x2 = x1
    y2 = y_center_main_way
    p2 = APoint(x2,y2)
    
    x3 = inverter_coor[0]
    y3 = y_center_main_way
    p3 = APoint(x3,y3)
    
    x4 = x3
    y4 = inverter_coor[1]
    p4 = APoint(x4,y4)

    line1 = acad.model.AddLine(p1,p2)
    line1.Color = 1
    line2 = acad.model.AddLine(p2,p3)
    line2.Color = 1
    line3 = acad.model.AddLine(p3,p4)
    line3.Color = 1

In [61]:
combine_zone[0][1] + walkway_width

1402963.6992474233

# Create Branch

In [62]:
#Walk Way create (Branch)
branch_way = []
x = combine_zone[1][0] + building_edge  #start point
y = yi + walkway_endclamp  #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x + walkway_width  #right point
y = y #right point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x #right bottom point
y = combine_zone[0][1] + walkway_width - 3*walkway_endclamp  #right bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x - walkway_width #right bottom point
y = y  #right bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = combine_zone[1][0] + building_edge  #start point
y = yi + walkway_endclamp #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

bw = aDouble(branch_way)
BranchWay = acad.model.AddPolyline(bw)
BranchWay.Color = 2

In [63]:
#Expand branch line

Pre_Walk = No_column_set*pv_length+(No_column_set-1)*pv_gap+walkway_pv_gap
Back_Walk = walkway_pv_gap + walkway_width
x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
Round = 0
pv = 0
m_point = Pre_Walk*(Round+1) + Round*Back_Walk
BranchWay.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"
x = x+m_point

#bw2 = BranchWay.Copy()
if PV_set%No_column_set!=0:
    PV_set_Round = PV_set-1
if PV_set%No_column_set==0:
    PV_set_Round = PV_set
    
while pv < PV_set_Round:
    #print("pv : "+str(pv))
    bw2 = BranchWay.Copy()
    m_point = (2*walkway_pv_gap + walkway_width + No_column_set*pv_length + (No_column_set-1)*pv_gap)*Round
    #m_point = Pre_Walk*(Round+1) + Round*Back_Walk
    pv = pv + No_column_set
    Round += 1
    
    bw2.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"

# Calculate DC cables